## DeepSynergy

Author: Kristina Preuer

This Keras script shows how DeepSynergy was evaluated in one cross validation run (executed 5 times - looping over test folds). In this examples fold 0 is used for testing. The script uses 60% of the data  for training (folds 2, 3, 4) and 20% for validation (fold 1). The parameters are loaded with a separate text file (hyperparameters). Validation loss was used to determine the early stopping parameter. After hyperparameter selection the training and validation data was combined (80% = folds 1, 2, 3, 4) and the remaining 20% (fold 0) of the data were used for testing.

The original work was done accordingly with binet (https://github.com/bioinf-jku/binet/tree/master/binet). 

In [3]:
import os, sys

import pandas as pd
import numpy as np
import pickle
import gzip

import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"]="3" #specify GPU 
from tensorflow import keras as K
import tensorflow as tf
from tensorflow.keras import backend
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

#### Define parameters for this cross-validation run

In [9]:
hyperparameter_file = 'hyperparameters' # textfile which contains the hyperparameters of the model
data_file = '/home/nidhi/Documents/freelancing/DeepSynergy/code/DeepSynergy/data_test_fold0_tanh.p.gz' # pickle file which contains the data (produced with normalize.ipynb)

#### Define smoothing functions for early stopping parameter

In [5]:
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

#### Load parameters defining the model

In [6]:
exec(open(hyperparameter_file).read()) 

#### Load data 
tr = 60% of data for training during hyperparameter selection <br>
val = 20% of data for validation during hyperparameter selection

train = tr + val = 80% of data for training during final testing <br>
test = remaining left out 20% of data for unbiased testing 

splitting and normalization was done with normalize.ipynb

In [10]:
file = gzip.open(data_file, 'rb')
X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test = pickle.load(file)
file.close()

#### run set

In [18]:
config = tf.compat.v1.ConfigProto(
         allow_soft_placement=True,
         #gpu_options = tf.GPUOptions(allow_growth=True)
        # gpu_options = tf.compat.v1.GPUOptions(allow_growth = True)
        )

config.gpu_options.allow_growth = True

In [21]:

#set_session(tf.compat.v1.Session()(config=config))

from tensorflow.compat.v1 import InteractiveSession
session = InteractiveSession(config=config)



In [22]:
model = Sequential()
for i in range(len(layers)):
    if i==0:
        model.add(Dense(layers[i], input_shape=(X_tr.shape[1],), activation=act_func, 
                        kernel_initializer='he_normal'))
        model.add(Dropout(float(input_dropout)))
    elif i==len(layers)-1:
        model.add(Dense(layers[i], activation='linear', kernel_initializer="he_normal"))
    else:
        model.add(Dense(layers[i], activation=act_func, kernel_initializer="he_normal"))
        model.add(Dropout(float(dropout)))
    model.compile(loss='mean_squared_error', optimizer=K.optimizers.SGD(lr=float(eta), momentum=0.5))

/home/nidhi/miniconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


#### run model for hyperparameter selection

In [24]:
hist = model.fit(X_tr, y_tr, epochs=2, shuffle=True, batch_size=64, validation_data=(X_val, y_val))
val_loss = hist.history['val_loss']
model.reset_states()

Epoch 1/2
434/434 [==============================] - 117s 269ms/step - loss: 533.2020 - val_loss: 417.1503
Epoch 2/2
434/434 [==============================] - 118s 272ms/step - loss: 478.9209 - val_loss: 383.9483


#### smooth validation loss for early stopping parameter determination

In [25]:
average_over = 15
mov_av = moving_average(np.array(val_loss), average_over)
smooth_val_loss = np.pad(mov_av, int(average_over/2), mode='edge')
epo = np.argmin(smooth_val_loss)

ValueError: can't extend empty axis 0 using modes other than 'constant' or 'empty'

#### determine model performance for methods comparison 

In [27]:
hist = model.fit(X_train, y_train, epochs=2, shuffle=True, batch_size=64, validation_data=(X_test, y_test))
test_loss = hist.history['val_loss']

Epoch 1/2
579/579 [==============================] - 154s 267ms/step - loss: 425.8561 - val_loss: 360.3245
Epoch 2/2
579/579 [==============================] - 158s 273ms/step - loss: 409.2623 - val_loss: 341.5284


#### plot performance 

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.plot(val_loss, label='validation loss')
ax.plot(smooth_val_loss, label='smooth validation loss')
ax.plot(test_loss, label='test loss')
ax.legend()
plt.show()